<a href="https://colab.research.google.com/github/doetools/deep_learning_tf/blob/main/deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

get started with basic tensorflow operations

In [4]:
print(tf.__version__)

2.12.0


the eager mode and has to use gradient tape
https://www.tensorflow.org/guide/autodiff

In [25]:
x = tf.Variable(3.0)
y = tf.Variable(4.0)

with tf.GradientTape() as tape:
  f = x*x*y + y +2

dy_dx = tape.gradient(f, x)
dy_dx.numpy()

24.0

the non-eager mode as decorated by tf.function, which will first convert the python function into a graph (DAG) and then solve it.

In [24]:
import tensorflow as tf

x = tf.Variable(3.,name="x",trainable=True)
y = tf.Variable(4.,name="y",trainable=True)


@tf.function
def fn(x, y):
  f = x*x*y + y +2
  g = tf.gradients(f,x)
  tf.print(g)
  return g

g = fn(x, y)
print(g)
print(type(g))


[24]
[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>]
<class 'list'>


some basic linear algbra


In [47]:
a = tf.constant([[1,2],[3 ,4]])
b = tf.constant([[3,4], [5, 6]])


mul= a*b

# a
tf.transpose(a)
a@tf.transpose(b)
mul == tf.multiply(a,b)

tf.reduce_sum(a)

if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

TensorFlow **IS** using the GPU


a linear regression example
p 236

In [110]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# m is row which is instance and n is column which is feature
m, n = housing.data.shape

# create the bais
bias = np.ones((m,1),dtype=np.float32)

# make a graph
x = tf.constant(housing.data, dtype=tf.float32)
bias = tf.constant(bias, dtype=tf.float32)
x = tf.concat([x,bias], 1)
y = tf.constant(housing.target,dtype=tf.float32)
y = tf.reshape(y, (-1,1))

# prepare the data
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_split = 0.8
train_ds = dataset.take(int(train_split*m))
test_ds = dataset.skip(int(train_split*m))

# create batch
dataset = dataset.shuffle(buffer_size=x.shape[0]*x.shape[1]).batch(batch_size)


# model
class Model(tf.Module):

  def __init__(self, n):
    # Randomly generate weight and bias terms
    rand_init = tf.random.uniform(shape=[n,1], minval=-1., maxval=1., seed=22)
    # Initialize model parameters
    self.w_l = tf.Variable(rand_init)

  @tf.function
  def __call__(self, x):
    return x @ self.w_l

linear_model = Model(n+1)


def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred,y))

### training
num_epochs = 1
learning_rate = 0.01
losses = []
for epoch in range(num_epochs):
  for x_batch, y_batch in dataset:
    with tf.GradientTape() as tape:
      y_pred = linear_model(x_batch)
      batch_loss = mse_loss(y_pred, y_batch)
      
    # gradient
    grads = tape.gradient(batch_loss, linear_model.variables)

    for g,v in zip(grads, linear_model.variables):
        print (g)
        v.assign_sub(learning_rate*g)
    break
  

  # loss
  loss = mse_loss(linear_model(x),y)
  losses.append(loss)
  print (loss.numpy())

  # print 
  if (epoch%10==0):
    print (loss.numpy())



tf.Tensor(
[[-7.3085366e+03]
 [-6.4782738e+04]
 [-1.0987012e+04]
 [-2.4991968e+03]
 [-4.9787200e+06]
 [-6.8137568e+03]
 [-8.2631227e+04]
 [ 2.7735919e+05]
 [-2.3190098e+03]], shape=(9, 1), dtype=float32)
8269147000000000.0
8269147000000000.0


In [61]:
import numpy as np

a = np.zeros((4,4))
b = np.ones((4,1))
a = tf.constant(a)
b = tf.constant(b)
tf.concat([a,b],1)

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])>

In [65]:
b = np.ones(10)
b
b.reshape((-1,1))

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [95]:

import numpy as np
import tensorflow as tf
 
N = 20   # number of samples
 
# Generate random samples roughly between -10 to +10
polynomial = np.poly1d([1, 2, 3])
X = np.random.randn(N,1) * 5
Y = polynomial(X)
 
# Prepare input as an array of shape (N,3)
XX = np.hstack([X*X, X, np.ones_like(X)])
 
# Prepare TensorFlow objects
w = tf.Variable(tf.random.normal((3,1)))  # the 3 coefficients
x = tf.constant(XX, dtype=tf.float32)     # input sample
w


<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-1.4239588 ],
       [-0.01400542],
       [ 0.64226186]], dtype=float32)>